# Spotify API Investigation

**Author:** Dermot O'Brien
***

## Data Understanding

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import base64
import requests
import datetime

%matplotlib inline

In [2]:
# Create variables for client id and client secret to create token
cid = 'fe9b61dd4aea4998a73a7e7f095bb66a'
secret = '8544bf1566dd4f3c8b5f80f032d9617e'

In [21]:
# Create a class for Spotify API
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    cid = None
    secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, cid, secret, *args, **kwargs):
        self.cid = cid
        self.secret = secret
        
    def get_client_credentials(self):
        cid = self.cid
        secret = self.secret
        if cid == None or secret == None:
            raise Exception("You must set client id and client secret")
        client_creds = f"{cid}:{secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def get_token_header(self):
        client_creds_b64 = self.get_client_credentials()
        return {'Authorization': f"Basic {client_creds_b64}"}
    
    def get_token_data(self):
        return {'grant_type': 'client_credentials'}
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_header = self.get_token_header()
        r = requests.post(token_url, data=token_data, headers=token_header)
        if r.status_code not in range (200, 299):
            return False
        data = r.json()
        access_token = data['access_token']
        now = datetime.datetime.now()
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

In [22]:
client = SpotifyAPI(cid, secret)

In [23]:
client.perform_auth()

True

In [25]:
access_token = client.access_token